<a href="https://colab.research.google.com/github/PacktPublishing/Modern-Computer-Vision-with-PyTorch-2E/blob/main/Chapter02/Specifying_batch_size_while_training_a_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch
import torch.nn as nn

In [2]:
x = [[1,2],[3,4],[5,6],[7,8]]
y = [[3],[7],[11],[15]]

In [3]:
X = torch.tensor(x).float()
Y = torch.tensor(y).float()

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
X = X.to(device)
Y = Y.to(device)

In [11]:
class MyDataset(Dataset):
    def __init__(self,x,y):
        self.x = torch.tensor(x).float()
        self.y = torch.tensor(y).float()
    def __len__(self):
        return len(self.x)
    def __getitem__(self, ix):
        return self.x[ix], self.y[ix]

ds = MyDataset(X, Y)

C:\Users\Aryan\AppData\Local\Temp\ipykernel_27416\2480197041.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.x = torch.tensor(x).float()
C:\Users\Aryan\AppData\Local\Temp\ipykernel_27416\2480197041.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.y = torch.tensor(y).float()


In [12]:
dl = DataLoader(ds, batch_size=2, shuffle=True)

In [13]:
class MyNeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.input_to_hidden_layer = nn.Linear(2,8)
        self.hidden_layer_activation = nn.ReLU()
        self.hidden_to_output_layer = nn.Linear(8,1)
    def forward(self, x):
        x = self.input_to_hidden_layer(x)
        x = self.hidden_layer_activation(x)
        x = self.hidden_to_output_layer(x)
        return x

In [79]:
mynet = MyNeuralNet().to(device)
loss_func = nn.MSELoss()
from torch.optim import SGD
opt = SGD(mynet.parameters(), lr = 0.001)

for x,y in dl: # 2*2 input --> 1*2 output
    print(x,y)

tensor([[7., 8.],
        [3., 4.]]) tensor([[15.],
        [ 7.]])
tensor([[5., 6.],
        [1., 2.]]) tensor([[11.],
        [ 3.]])


In [85]:
nonbatch_ds = TensorDataset(X,Y)
print(len(nonbatch_ds))
print(len(ds))

import time
loss_history = []
start = time.time()
for _ in range(50):
    for data in ds: #This is where we use the 2 instances/samples
        x, y = data
        opt.zero_grad() #This right here clears the previous instance of gradients; prevents gradient accumulation
        loss_value = loss_func(mynet(x),y)
        loss_value.backward()
        opt.step() #This is the update step (weight adjustment)
        loss_history.append(loss_value.item())
end = time.time()
print(end - start)

4
4
0.1966240406036377


In [86]:
val_x = [[10,11]]

In [87]:
val_x = torch.tensor(val_x).float().to(device)

In [88]:
mynet(val_x)

tensor([[21.1855]], grad_fn=<AddmmBackward0>)